In [303]:
import sqlite3
import pandas as pd
conn = sqlite3.connect("vivino.db")
cursor = conn.cursor()

In [304]:
cursor.execute('''SELECT AVG(vintages.ratings_count)
               FROM vintages
               LIMIT 10;''')
data = cursor.fetchall()
df1 = pd.DataFrame(data)
df1

,0
0,526.316585


## QUESTION 1 We want to highlight 10 wines to increase our sales. Which ones should we choose and why?

In [305]:
# We want to highlight 10 wines to increase our sales. Which ones should we choose and why?
# FOR VINTAGE WINES: HIGH PRIC
cursor.execute('''SELECT DISTINCT vintages.name, vintages.ratings_average, vintages.ratings_count, vintages.price_euros 
               FROM vintages 
               WHERE vintages.ratings_count > 500
               ORDER BY  vintages.price_euros DESC, vintages.ratings_average DESC,vintages.ratings_count DESC
               LIMIT 10;''')
data = cursor.fetchall()
cols = ['vintage_name', 'rating_average', 'rating_count', 'price_vintage']
df1 = pd.DataFrame(data, columns=cols)
df1['revenue_vintages'] = df1['rating_count'] * df1['price_vintage']
df1

,vintage_name,rating_average,rating_count,price_vintage,revenue_vintages
0,Pétrus Pomerol 1982,4.7,1361,9180.00,12493980.00
1,Pétrus Pomerol 1990,4.8,1190,6300.00,7497000.00
2,Château Lafite Rothschild Pauillac (Premier Gr...,4.7,1273,5977.50,7609357.50
3,Masseto Toscana 2001,4.8,651,5231.25,3405543.75
4,Masseto Toscana 2006,4.8,797,4910.00,3913270.00
5,Masseto Toscana 2005,4.8,762,4697.50,3579495.00
6,Masseto Toscana 2010,4.7,1054,4697.50,4951165.00
7,Pétrus Pomerol 1979,4.7,883,4410.00,3894030.00
8,Masseto Toscana 2004,4.7,767,4163.75,3193596.25
9,Masseto Toscana 2008,4.7,764,4163.75,3181105.00


In [306]:
import seaborn as sns
import plotly.express as px

df1_sorted = df1.sort_values(by='rating_count', ascending=False)
fig = px.bar(df1_sorted, x='vintage_name', y='rating_count', title='TOP 10: Total rating Count of exclusive Vintages')

fig.update_layout(
    yaxis_title='Rating Count',
    xaxis_title='Vintage Name',
    title_x=0.5
)

# fig.update_traces(text=df1['rating_count'], textposition='inside')
fig.update_traces(
    text=df1_sorted.apply(
        lambda row: f"{row['rating_count']}<br><span style='color: yellow;'>€ {row['price_vintage']:.2f} / unit</span>",
        axis=1
    ),
    textposition='inside'
)
# Show the plot
fig.show()

In [307]:
## QUESTION 1 - GRAPH 2

import plotly.express as px

df2_sorted = df1.sort_values(by='revenue_vintages', ascending=False)
fig = px.bar(df2_sorted, x='vintage_name', y='revenue_vintages', title='TOP 10: Highest revenue to date of highly rated exclusive Vintages')

fig.update_layout(
    yaxis_title='Total Revenue To Date (million €)',
    xaxis_title='Vintage Name',
    title_x=0.5
)

fig.update_traces(
    text=df2_sorted.apply(
        lambda row: f"€ {row['revenue_vintages']/1000000:.2f} m<br><span style='color: yellow;'>€ {row['price_vintage']:.2f} / unit</span>",
        axis=1
    ),
    textposition='inside'
)

# fig.update_traces(text=df2_sorted['revenue_vintages'].apply(lambda x: f"€ {x/1000000:.2f} m"), textposition='inside')

# Show the plot
fig.show()

In [308]:
## 'LOW END WINES' WITH RATINGS OVER 500

In [309]:
cursor.execute('''SELECT DISTINCT vintages.name, vintages.ratings_average, vintages.ratings_count, vintages.price_euros 
               FROM vintages 
               WHERE vintages.ratings_count > 500
               ORDER BY vintages.ratings_count DESC, vintages.ratings_average DESC
               LIMIT 10;''')
data = cursor.fetchall()
cols = ['vintage_name', 'rating_average', 'rating_count', 'price_vintage']
df3 = pd.DataFrame(data, columns=cols)
df3['revenue_vintages'] = df3['rating_count'] * df3['price_vintage']
df3

,vintage_name,rating_average,rating_count,price_vintage,revenue_vintages
0,Laurent-Perrier Brut Cuvée Champagne Rosé,4.4,36681,72.00,2641032.00
1,Krug Grande Cuvée,4.6,28513,245.00,6985685.00
2,Billecart-Salmon Brut Rosé Champagne,4.4,23241,81.50,1894141.50
3,Ruinart Brut Rosé Champagne,4.4,21147,49.61,1049102.67
4,Dom Pérignon Brut Champagne 2006,4.6,21121,440.00,9293240.00
5,San Marzano 60 Sessantanni Old Vines Primitivo...,4.5,17090,63.95,1092905.50
6,Opus One Overture,4.5,16804,180.00,3024720.00
7,Caymus Cabernet Sauvignon 2019,4.6,15647,177.95,2784383.65
8,Dom Pérignon Brut Champagne 2004,4.6,14650,460.00,6739000.00
9,Antinori Tignanello 2016,4.6,13709,420.00,5757780.00


In [310]:
merged_df = pd.concat([df1,df3]) 
sorted_df = merged_df.sort_values(by='revenue_vintages', ascending=False)
sorted_df.head(10)

,vintage_name,rating_average,rating_count,price_vintage,revenue_vintages
0,Pétrus Pomerol 1982,4.7,1361,9180.0,12493980.0
4,Dom Pérignon Brut Champagne 2006,4.6,21121,440.0,9293240.0
2,Château Lafite Rothschild Pauillac (Premier Gr...,4.7,1273,5977.5,7609357.5
1,Pétrus Pomerol 1990,4.8,1190,6300.0,7497000.0
1,Krug Grande Cuvée,4.6,28513,245.0,6985685.0
8,Dom Pérignon Brut Champagne 2004,4.6,14650,460.0,6739000.0
9,Antinori Tignanello 2016,4.6,13709,420.0,5757780.0
6,Masseto Toscana 2010,4.7,1054,4697.5,4951165.0
4,Masseto Toscana 2006,4.8,797,4910.0,3913270.0
7,Pétrus Pomerol 1979,4.7,883,4410.0,3894030.0


In [311]:
import plotly.express as px

sorted_df = sorted_df.head(10)
fig = px.bar(sorted_df, x='vintage_name', y='revenue_vintages', title='TOP 10: Highest revenue to date of highest rated wines (total ratings > 500)')

fig.update_layout(
    yaxis_title='Total revenue to date (million €)',
    xaxis_title='Vintage Name',
    title_x=0.5
)

fig.update_traces(
    text=sorted_df.apply(
        lambda row: f"€ {row['revenue_vintages']/1000000:.2f} m<br><span style='color: yellow;'>€ {row['price_vintage']:.2f} / unit</span>",
        axis=1
    ),
    textposition='inside'
)

# Show the plot
fig.show()

#### Question 2: What country should we focus on?

In [312]:
cursor.execute('''SELECT DISTINCT countries.name, vintages.name, vintages.ratings_average, vintages.ratings_count, vintages.price_euros 
               FROM wines
               JOIN vintages ON wine_id == wines.id
               JOIN regions ON region_id == regions.id
               JOIN countries ON regions.country_code == countries.code 
               WHERE vintages.ratings_count > 500
               ORDER BY  vintages.price_euros DESC, vintages.ratings_average DESC,vintages.ratings_count DESC
               LIMIT 10;''')
data = cursor.fetchall()
cols = ['country', 'vintage_name', 'rating_average', 'rating_count', 'price_vintage']
df3 = pd.DataFrame(data, columns=cols)
df3['revenue_vintages'] = df3['rating_count'] * df3['price_vintage']
df3 

,country,vintage_name,rating_average,rating_count,price_vintage,revenue_vintages
0,France,Pétrus Pomerol 1982,4.7,1361,9180.00,12493980.00
1,France,Pétrus Pomerol 1990,4.8,1190,6300.00,7497000.00
2,France,Château Lafite Rothschild Pauillac (Premier Gr...,4.7,1273,5977.50,7609357.50
3,Italie,Masseto Toscana 2001,4.8,651,5231.25,3405543.75
4,Italie,Masseto Toscana 2006,4.8,797,4910.00,3913270.00
5,Italie,Masseto Toscana 2005,4.8,762,4697.50,3579495.00
6,Italie,Masseto Toscana 2010,4.7,1054,4697.50,4951165.00
7,France,Pétrus Pomerol 1979,4.7,883,4410.00,3894030.00
8,Italie,Masseto Toscana 2004,4.7,767,4163.75,3193596.25
9,Italie,Masseto Toscana 2008,4.7,764,4163.75,3181105.00


In [313]:
df3_sorted = df3.sort_values(by='revenue_vintages', ascending=False)
fig = px.bar(df3_sorted, x='vintage_name', y='revenue_vintages', color='country', title='TOP 10: Highest revenue to date of highest rated wines (total ratings > 500)')

fig.update_layout(
    yaxis_title='Total Revenue To Date (million €)',
    xaxis_title='Vintage Name',
    title_x=0.5 # Adjust bottom margin to make space for text annotations
)

# fig.update_traces(text=df3_sorted['revenue_vintages'].apply(lambda x: f"€ {x/1000000:.2f} m"), textposition='inside')
for trace in fig.data:
    text_positions = []
    for i, (y, country) in enumerate(zip(trace.y, df3_sorted['country'])):
        if country == trace.name:
            text_positions.append(y)  # Adjust the value for proper placement
        else:
            text_positions.append(y)  # Adjust the value for proper placement
    trace.text = [f'€ {x/1000000:.2f} m' for x in text_positions]
    trace.textposition = 'inside'

fig.show()

In [314]:
# GRAPE TOP 3 + REASONING WRITE OUT 

# TOP 3 MOST COMMON GRAPES
cursor.execute( """SELECT most_used_grapes_per_country.grape_id, grapes.name, most_used_grapes_per_country.wines_count, COUNT(most_used_grapes_per_country.grape_id) AS grapeCount
             FROM most_used_grapes_per_country
             JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
             GROUP BY grape_id
             ORDER BY grapeCount DESC, wines_count DESC
             LIMIT 3
             """ )

cursor.fetchall() 

[(2, 'Cabernet Sauvignon', 801751, 12),
 (10, 'Merlot', 566719, 11),
 (5, 'Chardonnay', 604208, 6)]

#### REASONING WHY WE CANNOT LINK THE WINES TO THE GRAPES

- There is a clear link between the wines and the countries 

- wines.region_id --> regions.id > regions.country_code --> countries.code > most_used_grapes_per_country.country_code > grapes.name

Unfortunately we cannot link the wines to grapes name because the link ends at the "most_used_grapes_per_country" node. This only shows the most_used_grapes per country (every wine from that country will have these same grapes listed even though it would not necessarily mean that one of these grapes is used to produce the wine)

## BONUS QUESTION:
One of our VIP client likes Cabernet Sauvignon and would like our top 5 recommendations. Which wines would you recommend to him?

In [315]:
cursor.execute( """SELECT vintages.name, vintages.ratings_average, vintages.ratings_count,vintages.price_euros, vintage_toplists_rankings.rank
               FROM vintages
               JOIN vintage_toplists_rankings ON vintages.id = vintage_toplists_rankings.vintage_id
               JOIN wines ON wines.id = vintages.wine_id
               WHERE wines.name = 'Cabernet Sauvignon'
               ORDER BY vintages.ratings_count DESC, vintages.ratings_average DESC
               LIMIT 5
             """ )

cursor.fetchall() 

# DOES NOT SAY MUCH

[('Scarecrow Cabernet Sauvignon 2015', 4.8, 243, 1558.75, 1)]

In [316]:
cursor.execute( """SELECT vintages.name, vintages.ratings_average, vintages.ratings_count, vintages.price_euros
               FROM vintages
               JOIN wines ON wines.id = vintages.wine_id
               WHERE wines.name = 'Cabernet Sauvignon'
               AND vintages.ratings_average > 4.5
               ORDER BY vintages.price_euros DESC, vintages.ratings_average DESC,  vintages.ratings_count DESC
               LIMIT 5
             """ )

data = cursor.fetchall() 

# SCARECROW 2015 IS RANKED NUMBER ONE 
# ------------------------------------------------------------------------------------------------------------------------

import pandas as pd
import plotly.express as px

# Create a DataFrame from the sample data

df = pd.DataFrame(data, columns=['Name', 'Rating', 'Rating Count', 'Price'])

# Sort the DataFrame by Rating Count in descending order
df = df.sort_values(by='Rating Count', ascending=False)

# Create a bar plot using Plotly Express
fig = px.bar(
    df,
    x='Name',
    y='Rating Count',
    color='Rating',
    hover_data=['Price'],
    text=df['Price'].apply(lambda x: f'€ {x:.2f} / unit'),  # Display the formatted price on the bars
    title='Top Cabernet Sauvignon by Rating',
    labels={'Rating Count': 'Rating Count', 'Name': 'Wine Name'}
)

# Rotate x-axis labels for better readability
fig.update_xaxes(tickangle=-45)

# Update the positioning of the price text on the bars
fig.update_traces(textposition='inside')

# Show the plot
fig.show()

In [317]:
test = '''
SELECT 
    GROUP_CONCAT(DISTINCT keywords.name) AS mixed_tastes,
    GROUP_CONCAT(DISTINCT keywords_wine.group_name) AS taste_group,
    COUNT(DISTINCT keywords.name) AS count_of_tastes
FROM 
    wines
JOIN 
    regions ON wines.region_id = regions.id
JOIN 
    countries ON regions.country_code = countries.code
JOIN 
    keywords_wine ON wines.id = keywords_wine.wine_id
JOIN 
    keywords ON keywords_wine.keyword_id = keywords.id
WHERE 
    keywords.name IN ('coffee', 'toast', 'green apple', 'cream', 'citrus')
AND 
    keywords_wine.count > 10  -- More than 10 users confirmed the keywords
AND 
    keywords_wine.keyword_type = 'primary'
GROUP BY
    wines.name
HAVING 
    COUNT(DISTINCT keywords.name) >= 2
ORDER BY count_of_tastes DESC, mixed_tastes DESC
'''
cursor.execute(test)
data = cursor.fetchall()

cols = ['mixed_tastes', 'taste_groups', 'count_mixed_tastes']
df_13 = pd.DataFrame(data, columns=cols)
# pd.set_option('display.max_rows', None)  # Display all rows
df_13.head(3)

,mixed_tastes,taste_groups,count_mixed_tastes
0,"toast,green apple,citrus,coffee,cream","non_oak,tree_fruit,citrus_fruit,oak,microbio",5
1,"toast,cream,coffee,green apple,citrus","non_oak,microbio,oak,tree_fruit,citrus_fruit",5
2,"toast,cream,coffee,green apple,citrus","non_oak,microbio,oak,tree_fruit,citrus_fruit",5


In [318]:
## QUESTIONS 4-5

import pandas as pd
import plotly.express as px

# Assuming you already have the DataFrame named df
# Calculate the percentage of items for each count
count_percentage = df_13['taste_groups'].value_counts(normalize=True) * 100

# Filter labels and percentages for slices with percentage > 2.5%
filtered_data = count_percentage[count_percentage > 2.5]

# Create a DataFrame for the filtered data
filtered_df = pd.DataFrame({'Taste Groups': filtered_data.index, 'Percentage': filtered_data.values})

# Create a pie chart using Plotly Express
fig = px.pie(
    data_frame=filtered_df,
    values='Percentage',
    names='Taste Groups',
    title='Percentage of each taste group',
    labels={'Taste Groups': 'Taste Groups'},
    hover_data={'Percentage': ':.1f%'}
)

# Center the legend to the right of the pie chart
fig.update_layout(legend_title_text='Taste Groups', legend=dict(orientation="v", yanchor="middle", y=0.5, xanchor="left", x=1))

# Show the plot
fig.show()

In [319]:
# import pandas as pd
# import matplotlib.pyplot as plt

# # Assuming you already have the DataFrame named df
# # Calculate the percentage of items for each count
# count_percentage = df_13['count_mixed_tastes'].value_counts(normalize=True) * 100

# # Create a bar plot
# plt.bar(count_percentage.index, count_percentage.values)
# plt.xlabel('Count of Tastes')
# plt.ylabel('Percentage')
# plt.title('Percentage of Items for Each Count of Tastes')
# plt.show()

In [320]:
# import pandas as pd
# import matplotlib.pyplot as plt

# # Assuming you already have the DataFrame named df
# # Calculate the percentage of items for each count
# count_percentage = df_13['mixed_tastes'].value_counts(normalize=True) * 100

# # Create a bar plot
# plt.bar(count_percentage.index, count_percentage.values)
# plt.xlabel('Count of Tastes')
# plt.xticks(rotation=90)
# plt.ylabel('Percentage')
# plt.title('Percentage of Items for Each Count of Tastes')
# plt.show()

In [321]:
# import pandas as pd
# import matplotlib.pyplot as plt

# # Assuming you already have the DataFrame named df
# # Calculate the percentage of items for each count
# count_percentage = df_13['taste_groups'].value_counts(normalize=True) * 100

# # Create a bar plot
# plt.bar(count_percentage.index, count_percentage.values)
# plt.xlabel('Count of Tastes')
# plt.xticks(rotation=90)
# plt.ylabel('Percentage')
# plt.title('Percentage of Items for Each Count of Tastes')
# plt.show()